In [1]:
import urllib.request
import zipfile
import re

# 青空文庫のURLからテキストを取得する関数
def get_text_from_aozora_url(url):
    # zipファイルのダウンロード
    zip = re.split(r'/', url)[-1]
    urllib.request.urlretrieve(url, zip)

    # zipファイルの解凍
    with zipfile.ZipFile(zip, 'r') as myzip:
        myzip.extractall()
        for myfile in myzip.infolist():
            filename = myfile.filename
            with open(filename, encoding='sjis') as file:
                text_org = file.read()

    # テキスト整形
    text = re.split('\-{5,}',text_org)[2]
    text = re.split('底本：',text)[0]
    text = re.sub('《.+?》', '', text)
    text = re.sub('［＃.+?］', '',text)
    text = re.sub('\n\n', '\n', text) 
    text = re.sub('\r', '', text)
    text = re.sub("[｜ 　]", "", text)
    
    return text

In [2]:
import time

# ダウンロードURLリスト
url_list = ['https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip',
            'https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip',
            'https://www.aozora.gr.jp/cards/000148/files/752_ruby_2438.zip'
]

merge_text = ""
for url in url_list:
    # テキスト取得
    text = get_text_from_aozora_url(url)
    # サーバーに負荷をかけないよう1秒空ける
    time.sleep(1)
    # ダウンロードしたテキストのマージ
    merge_text += text + ("\n")

# print(merge_text)

In [3]:
file_path = "train.txt"
with open(file_path, mode="w") as f:
    f.write(merge_text)

In [4]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 33.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 163 kB 70.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 31.4 MB/s 


In [5]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

Downloading:   0%|          | 0.00/806k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

In [6]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

# 訓練用データセットの設定
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=file_path,
    block_size=128
)

# データ入力の設定
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 訓練の設定
training_args = TrainingArguments(
    output_dir="./gpt2-ft-soseki",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

# 訓練の実施
trainer.train()

# 訓練済みモデルの保存
trainer.save_model("./souseki_model")


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1932
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 726


Step,Training Loss
500,3.610900


Saving model checkpoint to ./gpt2-ft-soseki/checkpoint-500
Configuration saved in ./gpt2-ft-soseki/checkpoint-500/config.json
Model weights saved in ./gpt2-ft-soseki/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./souseki_model
Configuration saved in ./souseki_model/config.json
Model weights saved in ./souseki_model/pytorch_model.bin


In [7]:
def getarate_sentence(seed_sentence):
    x = tokenizer.encode(seed_sentence, return_tensors="pt", add_special_tokens=False)
    x = x.cuda()
    y = model.generate(input_ids=x, 
                       min_length=256, 
                       max_length=512, 
                       do_sample=True, 
                       pad_token_id=tokenizer.pad_token_id, 
                       bos_token_id=tokenizer.bos_token_id, 
                       eos_token_id=tokenizer.eos_token_id
                       )
    generated_sentence = tokenizer.batch_decode(y, skip_special_tokens=True)
    return generated_sentence

In [11]:
seed_sentence = "メロスは激怒した。"
generated_sentence = getarate_sentence(seed_sentence)
print(generated_sentence)

['メロスは激怒した。 「君のお嬢さんはもうこの世にはいらんんだな」と言いながら、は少し笑った。 「そうでもありませんよ」 「それじゃもうこの先どうするかまだいいません」 「君はいつ卒業するんですか」 「ええ、まだです」 「そう。二十歳になってください」 「そりゃ卒業は早くしてもいいじゃありませんか」 「それは学校じゃいわれます」 「その学校はどこですか。そこをあたってみましょう」 「今度いくらくらいかかるんでしょう」 奥さんはもうどこにいるかわからない様子であった。やがてまた静かに話し始めた。与次郎と奥さんはよく聞くと無口 [ [続きを読む] 私は長い間郷里を離れて暮 [続きを読む] 私は叔父の奥さんに、こう尋ねました。「私は今までずっと奥 [続きを読む] 十三 「この春、お嬢さんが卒業式の時の作文を東京へ持ってゆくのはどう[続きを読む] 私は親戚の伯父さんに会った。叔父は東京の 大学の病院のそばに家を持ったお父さんと、そのお母さ[続きを読む] 母は母の妹の縁談を承りました。その人の [続きを読む]']
